# Day 3: Disney Parks Guest Spending Behavior

You are a data analyst working with the Disney Parks revenue team to understand nuanced guest spending patterns across different park experiences. The team wants to develop a comprehensive view of visitor purchasing behaviors. Your goal is to uncover meaningful insights that can drive personalized marketing strategies.

In [ ]:
import pandas as pd
import numpy as np

fct_guest_spending_data = [
  {
    "guest_id": 1,
    "visit_date": "2024-07-05",
    "amount_spent": 50,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-07-06",
    "amount_spent": 30,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-07-10",
    "amount_spent": 20.5,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 4,
    "visit_date": "2024-07-12",
    "amount_spent": 40,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-07-15",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 5,
    "visit_date": "2024-07-20",
    "amount_spent": 60,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 6,
    "visit_date": "2024-07-25",
    "amount_spent": 25,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-03",
    "amount_spent": 55,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-15",
    "amount_spent": 45,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-05",
    "amount_spent": 22,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-20",
    "amount_spent": 38,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 7,
    "visit_date": "2024-08-10",
    "amount_spent": 15,
    "park_experience_type": "Character Meet"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-25",
    "amount_spent": 28,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-27",
    "amount_spent": 32,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-02",
    "amount_spent": 65,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-05",
    "amount_spent": 50,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 9,
    "visit_date": "2024-09-15",
    "amount_spent": 40,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 10,
    "visit_date": "2024-09-20",
    "amount_spent": 70,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-25",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-28",
    "amount_spent": 10,
    "park_experience_type": "Character Meet"
  }
]
fct_guest_spending = pd.DataFrame(fct_guest_spending_data)


## Question 1

What is the average spending per guest per visit for each park experience type during July 2024? Ensure that park experience types with no recorded transactions are shown with an average spending of 0.0. This analysis helps establish baseline spending differences essential for later segmentation.

In [ ]:
# Note: pandas and numpy are already imported as pd and np
# The following tables are loaded as pandas DataFrames with the same names: fct_guest_spending
# Please print your final result or dataframe
# Filter for July
july_data = fct_guest_spending.query("visit_date >= '2024-07-01' and visit_date < '2024-08-01'")

# Calculate total spending and total visits per park experience type over all guests
total_spending_visits = july_data.groupby('park_experience_type').agg(
    total_spent=('amount_spent', 'sum'),
    total_visits=('amount_spent', 'count')
).reset_index()

# Compute average spending per visit per experience
total_spending_visits['avg_spending_per_visit'] = total_spending_visits['total_spent'] / total_spending_visits['total_visits']

# Merge with all experience types to include types with zero visits (fill with 0.0)
all_experience_types_df = fct_guest_spending[['park_experience_type']].drop_duplicates()
result = pd.merge(all_experience_types_df, total_spending_visits[['park_experience_type', 'avg_spending_per_visit']],
                  on='park_experience_type', how='left')
result['avg_spending_per_visit'] = result['avg_spending_per_visit'].fillna(0.0)

print(result.rename(columns={'avg_spending_per_visit': 'avg_spending_per_guest_visit'}))

## Question 2

For guests who visited our parks more than once in August 2024, what is the difference in spending between their first and their last visit? This investigation, using sequential analysis, will reveal any shifts in guest spending behavior over multiple visits.

In [ ]:
# Filter for August 2024 visits
august_data = fct_guest_spending.query("visit_date >= '2024-08-01' and visit_date < '2024-09-01'")

# Sort by guest_id and visit_date to get visits in order
august_data_sorted = august_data.sort_values(['guest_id', 'visit_date'])

first_visits = august_data_sorted.groupby('guest_id').first().reset_index()
last_visits = august_data_sorted.groupby('guest_id').last().reset_index()

# Merge first and last visits on guest_id
visits_compare = pd.merge(
    first_visits[['guest_id', 'amount_spent']],
    last_visits[['guest_id', 'amount_spent']],
    on='guest_id',
    suffixes=('_first', '_last')
)

# Filter to guests with more than one visit
visit_counts = august_data_sorted.groupby('guest_id').size().reset_index(name='visit_count')
visits_compare = visits_compare.merge(visit_counts, on='guest_id')
visits_compare = visits_compare[visits_compare['visit_count'] > 1]

# Calculate difference: last visit spending - first visit spending
visits_compare['spending_difference'] = visits_compare['amount_spent_last'] - visits_compare['amount_spent_first']

print(visits_compare[['guest_id', 'spending_difference']])

## Question 3

In September 2024, how can guests be categorized into distinct spending segments such as Low, Medium, and High based on their total spending? Use the following thresholds for categorization: 
-Low: Includes values from $0 up to, but not including, $50.
-Medium: Includes values from $50 up to, but not including, $100.
-High: Includes values from $100 and above. 
Exclude guests who did not make any purchases in the period.

In [ ]:
# Filter for September 2024
sep_data = fct_guest_spending.query("visit_date >= '2024-09-01' and visit_date < '2024-10-01'")

# Aggregate total spending per guest in September
guest_spending_sep = sep_data.groupby('guest_id')['amount_spent'].sum().reset_index()

# Exclude guests with zero spending
guest_spending_sep = guest_spending_sep[guest_spending_sep['amount_spent'] > 0]

# Define spending bins and labels according to your thresholds
bins = [0, 50, 100, float('inf')]  
labels = ['Low', 'Medium', 'High']

# Categorize total spending into segments
guest_spending_sep['spending_segment'] = pd.cut(
    guest_spending_sep['amount_spent'],
    bins=bins,
    labels=labels,
    right=False,  # intervals are left-inclusive, right-exclusive
    include_lowest=True
)

print(guest_spending_sep[['guest_id', 'amount_spent', 'spending_segment']])

Made with ❤️ by [Interview Master](https://www.interviewmaster.ai)